In [10]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [11]:
#Levanto tabla de ramos

ramos_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TÉCNICA\Python Caja Negra\input\tabla_ramos.xlsx', index_col = 'cod_subramo')

ANEXO 12 - PRIMAS

In [12]:
# Anexo 12 Primas - el primer trimestre tiene que ser 3

anexo12_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TÉCNICA\Python Caja Negra\input\anexo12.xlsx')

anexo12_df.cod_cia=anexo12_df.cod_cia.astype('O')

In [14]:
# Filtro por los sub-ramos de Riesgos del trabajo

anexo12_df=anexo12_df[(anexo12_df.cod_subramo=='1.050.01')|(anexo12_df.cod_subramo=='1.050.99')]

In [15]:
# Crea la columna Ramo y la completa buscando la información de la tabla de ramos

anexo12_df['ramo']=[ramos_df.loc[anexo12_df.cod_subramo[i],'des_ramo'] for i in anexo12_df.index]

In [16]:
# Crea una tabla del tipo "tabla dinámica de excel"
# Agrupa filas: por ID y ramo, columnas: año (año=aaaa-t) y suma las primas emitidas

anexo12_pivot=anexo12_df.pivot_table(index=['cod_cia','ramo'],values=['Primas_Emitidas'], columns=['año'], aggfunc=np.sum)

In [19]:
# Reemplaza los missing por nulos

anexo12_pivot=anexo12_pivot.replace(to_replace=np.nan, value=0)

In [20]:
#Creación base trimestral desacumulada

# Creo el esqueleto donde después va a incluir la información con un bucle. Lo que le estoy pidiendo es que utilice como esqueleto
# la misma estructura que tengo hasta ahora en 'anexo12_pivot' y que le modifique el nombre de las columnas a partir de 'multi_index'


multi_index=pd.MultiIndex(levels=[list(anexo12_pivot.columns.levels[0]), list(anexo12_pivot.columns.levels[1])],
                                    labels=list(anexo12_pivot.columns.labels),
                                    names=['variable', 'año'])


anexo12_trim=pd.DataFrame(index=anexo12_pivot.index , columns=multi_index)


In [21]:
# Defino la función que voy a utilizar para hacer el cálculo

# La función indica que si el período de análidis es 3, entonces que retorne el mismo monto de prima emitida. 
# Sino que al valor del período actual le reste el período anterior.

def trim(row,col,año):
    if anexo12_trim.loc[anexo12_trim.iloc[row].name[0]][col][año].name.split('-')[1]=='3':
        
        return anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][año]
    
    else:
        
        return anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][año]-anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][anexo12_trim.columns.levels[1][list(anexo12_trim.columns.levels[1]).index(año)-1]]
            

In [22]:
# Defino el bucle for al que voy a aplicar la función

variables=['Primas_Emitidas']

for j in range(len(anexo12_trim.index)):
    for col in variables:
        for año in list(anexo12_trim.columns.levels[1]):
            anexo12_trim.iloc[j][col][año]=trim(j,col,año)

In [23]:
anexo12_trim['cod']=[anexo12_trim.iloc[i].name[0] for i in range(len(anexo12_trim.index))]
anexo12_trim['ramo']=[anexo12_trim.iloc[i].name[1] for i in range(len(anexo12_trim.index))]

In [24]:
anexo12_trim_melt=anexo12_trim.melt(id_vars=['cod','ramo'])

In [26]:
anexo12_trim_melt.to_excel(r'C:\Users\eangenello\Desktop\TÉCNICA\Python Caja Negra\output\anexo12_desacumulado.xlsx')